In [72]:
library(ggplot2)
library(reshape2)
library(dplyr)
library(ggpubr)
library('ggplot2')
library('reshape2')
library('ggpubr')
library(glmnet)
library(doMC)
library(survival)
library(data.table)
library(mltools)
library(CoxBoost)
library(randomForestSRC)
library(CoxHD)
library(Hmisc)
library(gridExtra)
library("survminer")
library(dplyr)
library(stringr)
source("../../../../src/tools.R")
options(repr.plot.res        = 100, # set a medium-definition resolution for the jupyter notebooks plots (DPI)
        repr.matrix.max.rows = 250, # set the maximum number of rows displayed
        repr.matrix.max.cols = 250)

In [73]:
colors <- c("Components" = "#EE9937", "ELN Recommendation 2017" = "#5C5C5C", "comp" = "#870C14", "eln_comp" = "#BFBFBF",
         "eln_gen" = "#59A08B", "eln_cyto" = "#2b8cbe", "eln_gen_cyto" = "#a6bddb", "eln_gen_cyto_comp" = "#fdbb84",
         "comp_gen" = "grey45", "comp_cyto" = "#e79f00", "comp_gen_cyto" = "#009E73", "cyto" = "#0072B2",
         "gen" = "#CC79A7", "gen_cyto" = "#9ad0f3", "eln_age" = "#D55E00", "eln_demo" = "lightskyblue","eln_clin" = "mistyrose4", "eln_clin_demo" = "darkseagreen","comp_age" = "lightgoldenrod","comp_clin" = "lightsalmon","comp_demo" = "#F0E442","comp_clin_demo" = "firebrick3","comps" = "#000000")
is.element("Components",names(colors))

[1] TRUE

In [77]:
SVM <- read.table("SVM_validation.csv",sep=',',header=T)

In [81]:


comparison <-function(models = list(c("eln","comp")), remove_RFX_ = F ,x = 25, y = 8 ,tests="",color="",FUN=mean ){
       options(warn=-1)
    with_colors = T
    df_comparison <- cbind(read.table(paste(paste("Prognosis/",models[1],sep=""),".tsv",sep="")),SVM=SVM[,models[1]])
    df_comparison$model <- ifelse(models[1]=="eln","ELN Recommendation 2017",
                         ifelse(models[1]=="comp","Components",models[1]))
    for (mod in models[-1]){
        tmp <- cbind(read.table(paste(paste("Prognosis/",mod,sep=""),".tsv",sep="")),SVM=SVM[,mod])
        tmp$model <- ifelse(mod=="eln","ELN Recommendation 2017",
                         ifelse(mod=="comp","Components",mod))
        if(with_colors){
                with_colors = is.element(unique(tmp$model),names(colors))
        }
        df_comparison <- rbind(df_comparison,tmp)
    }
    df_comparison <- melt(df_comparison,id='model')
    if (remove_RFX_){df_comparison <- df_comparison[df_comparison$variable!= "RFX_",]}
    df_comparison $title <- "Detailed Comparison NEJM Validation"
    df_comparison$title_bis <- "Summary Comparison NEJM Validation"
    
    
    
    
    set_notebook_plot_size(x,y)
    p1 <- ggplot(df_comparison,aes(x=variable,y=value)) + geom_boxplot(aes(fill = model), notch=F)+
    theme(strip.text = element_text(size=30),legend.position="top",axis.text.x = element_text(angle = 35, hjust = 0.8,vjust=0.8,size=20),axis.text.y = element_text(size = 20),
         axis.title.x=element_text(size=30),axis.title.y=element_text(size=30),legend.title=element_text(size=30),legend.text=element_text(size=20)) +
    ylab("C-Index") + xlab("") + facet_wrap( ~ title) + try(if(length(color)>1){scale_fill_manual(values=color)})
    
    p2 <- ggplot(df_comparison, aes(x=reorder(model,-value,FUN=FUN),y=value))+
    geom_boxplot(aes(fill =model), notch=F)+
    theme(strip.text = element_text(size=30),legend.position="none",axis.text.x = element_text(angle = 35, hjust = 0.8,vjust=0.8,size=15),axis.text.y = element_text(size = 15),
             axis.title.x=element_text(size=30),axis.title.y=element_text(size=30),legend.title=element_text(size=30),legend.text=element_text(size=20))+
    #
     stat_compare_means(paired=F,comparison=tests,method="wilcox.test")+
#              symnum.args = list(cutpoints = c(0, 0.0000001, 0.000001, 0.00001,0.0001, 0.001, 0.05), symbols = c("*****","****", "***", "**", "*", "ns")))+
             #,+
    xlab("")+ylab("C-Index")+ facet_wrap( ~ title_bis) + try(if(length(color)>1) {scale_fill_manual(values=color)}) + scale_y_continuous(position = "right")
    return(list(p1,p2))
}

# FIGURES

# FIGURE : Impact on C-I of the different correlates

In [82]:
# pdf("../graphs/Model_Added_value/evolution_summary.pdf",width=25,height=8)
png("../../../script_figures/val_evolution_summary.png",width=4000,height=2000,res=200)
comparison(c("eln_comp_gen_cyto_clin_demo","gen","cyto","clin","demo","gen_cyto","gen_cyto_clin_demo","cyto_gen_demo","comp","eln","comp_clin_demo"),tests=list(c("gen","cyto"),c("gen","gen_cyto"),c("cyto","gen_cyto"),
c("gen_cyto","cyto_gen_demo"),c("cyto_gen_demo","gen_cyto_clin_demo"),c("Components","gen_cyto"),c("gen_cyto_clin_demo","comp_clin_demo")
,c("comp_clin_demo","eln_comp_gen_cyto_clin_demo")),
          color=c("ELN Recommendation 2017"="#5C5C5C","Components"="#59A08B","gen"="#BFBFBF","gen_cyto"="#EE9937","gen_cyto_clin_demo"="mistyrose4","cyto"="#2b8cbe","clin"="#870C14","demo"="#a6bddb",
                  "cyto_gen_demo"="lightgoldenrod","comp_clin_demo"="firebrick3","eln_comp_gen_cyto_clin_demo"="lightsalmon"))
dev.off()

[[1]]

[[2]]


pdf 
  2